In [1]:
import numpy as np
import scipy
import scipy.io

In [2]:
sax_rate, sax = scipy.io.wavfile.read('sax.wav')
mix_rate, mix = scipy.io.wavfile.read('mix.wav')

In [3]:
mono_sax = np.empty((250000),np.float32)
mono_mix = np.empty((250000),np.float32)
for i in range(mono_sax.size):
    mono_sax[i] = round((sax[i][0]+sax[i][1])/2)

In [4]:
class My_Scaler:
    def __init__(self, min, max, a, b):
        self.a = a
        self.b = b
        self.min = min
        self.max = max
    
    def transform(self, input):
        return ((self.b-self.a)*((input-self.min)/(self.max-self.min)))+self.a
    
    def inverse_transform(self,input):
        return ((self.max - self.min)*((input+self.b)/(self.b-self.a))+self.min)

In [5]:
min = mono_mix.min()
if mono_sax.min() < min:
    min = mono_sax.min()
max = mono_mix.max()
if mono_sax.max() > max:
    max = mono_sax.max()
scaler = My_Scaler(min,max,-1,1)

In [6]:
x = scaler.transform(mono_mix)
y = scaler.transform(mono_sax)

In [7]:
encoding = 256
seq_len = 128
batch_size = 512

In [8]:
data = np.empty([250000,2],np.float32)

for i in range(data.shape[0]):
    data[i][0] = x[i]
    data[i][1] = y[i]

In [9]:
import torch
from torchaudio.functional import mu_law_encoding, mu_law_decoding
data=mu_law_encoding(torch.from_numpy(data),encoding)

In [10]:
class TimesereisDataset(torch.utils.data.Dataset):
    def __init__(self, data, seq_len=128):
        super().__init__()
        self.data=data
        self.seq_len=seq_len
    
    def __len__(self):
        return 25000-(self.seq_len+1)
    
    def __getitem__(self,index):
        x=self.data[index:index+self.seq_len,0]
        y=self.data[index+self.seq_len,1]
        return x,y  

In [11]:
data = data.numpy()

In [12]:
one_hot_data = []
for i in range(data.shape[0]):
    x_hot = np.zeros(encoding)
    y_hot = np.zeros(encoding)
    x_hot[data[i][0]] = 1.0
    y_hot[data[i][1]] = 1.0
    one_hot_data.append((x_hot,y_hot))
one_hot_data = np.array(one_hot_data)

In [13]:
timeSeriesDataset = TimesereisDataset(one_hot_data,seq_len)

In [14]:
train = []
test=[]
for i in range(len(timeSeriesDataset)):
    if i%10 < 8:
        train.append(timeSeriesDataset[i])
    else: test.append(timeSeriesDataset[i])

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [16]:
model=Sequential()
model.add(LSTM(input_shape=(128,256)))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid',))

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
xtrain = train[:][0]
ytrain = train[:][1][0]

xtrain = np.array(xtrain, dtype=np.float32)
ytrain = np.array(ytrain,dtype=np.float32)

print(ytrain[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [19]:
model.fit(xtrain, ytrain, epochs=3, batch_size=1)

Epoch 1/3


ValueError: in user code:

    File "C:\Users\Kevin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Kevin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Kevin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Kevin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py", line